In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'Info_Regions.csv', 'Train_mobile_500_Conso.csv', 'Test_Regions.csv', 'training.csv', 'Train_Regions.csv', 'Train_to_banks_Conso.csv', 'Test_to_banks.csv', 'Test_mobile_400.csv']


In [2]:
# Import libraries
import pandas as pd
import scipy as scipy
from scipy import io
import matplotlib.pyplot as plt
import seaborn as sns

# For map plotting
import folium
import os
import numpy as np
import bokeh
import geopandas as gpd
from branca.colormap import LinearColormap
from folium.plugins import FastMarkerCluster
from folium import plugins

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from tpot import TPOTClassifier
from sklearn.metrics import log_loss,accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import *
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
# Import modules
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from matplotlib.colors import ListedColormap

In [3]:
# Loading files
train = pd.read_csv('../input/training.csv',decimal=".")
# the test set is the test for the predictions on the plateform. 
# But for training the model we will need to divide the training set in two : Train and Test
# We will thus rename this data set "to_predict" to clarify
to_predict = pd.read_csv('../input/test.csv',decimal=".")

In [4]:
# Extract from ArcGis on the distance of each point with the closest bank
distances_to_bank = pd.read_csv('../input/Train_to_banks_Conso.csv',decimal=".")
# remove duplicates in the distances_to_bank file keeping only the closest banks
distances_to_bank_reduced = distances_to_bank.sort_values('Total_Kilometers')
distances_to_bank_reduced = distances_to_bank_reduced.drop_duplicates(subset='From_ID_Orig', keep='first')
# merging with the training dataset
distances_final = distances_to_bank_reduced[['From_ID_Orig','Total_Kilometers']]
distances_final.columns = ['ID','KM_to_bank']
train = pd.merge(train, distances_final, on=['ID'],how='left')
# remplacing the NaN values by 100km (the filter in Arcgis was 100km and above)
train = train.fillna(100)
# remplacing all values above 100km by 100km
train['KM_to_bank'] = train['KM_to_bank'].apply(lambda x : 100 if x>100 else x)

In [5]:
# Extract from ArcGis on the distance of each point with the closest mobile_money_vendor
distances_to_mobile = pd.read_csv('../input/Train_mobile_500_Conso.csv',decimal=".")

In [6]:
# remove duplicates in the distances_to_mobile file keeping only the closest mobile money vendors
distances_to_mobile_reduced = distances_to_mobile.sort_values('Total_Kilometers')
distances_to_mobile_reduced = distances_to_mobile_reduced.drop_duplicates(subset='From_ID_Orig', keep='first')
# merging with the training dataset
distances_to_mobile_final = distances_to_mobile_reduced[['From_ID_Orig','Total_Kilometers']]
distances_to_mobile_final.columns = ['ID','KM_to_mobile']
train = pd.merge(train, distances_to_mobile_final, on=['ID'],how='left')
# remplacing the NaN values by 400km (the filter in Arcgis was 400km and above for the test set)
train = train.fillna(400)
# remplacing all values above 400km by 400km
train['KM_to_mobile'] = train['KM_to_mobile'].apply(lambda x : 400 if x>400 else x)

In [7]:
# Extract from ArcGis on the distance of each point with the closest bank
distances_to_bank_to_predict = pd.read_csv('../input/Test_to_banks.csv',decimal=".")
# remove duplicates in the distances_to_bank file keeping only the closest banks
distances_to_bank_reduced_to_predict = distances_to_bank_to_predict.sort_values('Total_Kilometers')
distances_to_bank_reduced_to_predict = distances_to_bank_reduced_to_predict.drop_duplicates(subset='From_ID_Orig', keep='first')
# merging with the training dataset
distances_final_to_predict = distances_to_bank_reduced_to_predict[['From_ID_Orig','Total_Kilometers']]
distances_final_to_predict.columns = ['ID','KM_to_bank']
to_predict = pd.merge(to_predict, distances_final_to_predict, on=['ID'],how='left')
# remplacing the NaN values by 100km (the filter in Arcgis was 100km and above)
to_predict = to_predict.fillna(100)
# remplacing all values above 100km by 100km
to_predict['KM_to_bank'] = to_predict['KM_to_bank'].apply(lambda x : 100 if x>100 else x)

In [8]:
# Extract from ArcGis on the distance of each point with the closest mobile_money_vendor
distances_to_mobile_to_predict = pd.read_csv('../input/Test_mobile_400.csv',decimal=".")

In [9]:
# remove duplicates in the distances_to_mobile file keeping only the closest mobile money vendors
distances_to_mobile_reduced_to_predict = distances_to_mobile_to_predict.sort_values('Total_Kilometers')
distances_to_mobile_reduced_to_predict = distances_to_mobile_reduced_to_predict.drop_duplicates(subset='From_ID_Orig', keep='first')
# merging with the training dataset
distances_to_mobile_final_to_predict = distances_to_mobile_reduced_to_predict[['From_ID_Orig','Total_Kilometers']]
distances_to_mobile_final_to_predict.columns = ['ID','KM_to_mobile']
to_predict = pd.merge(to_predict, distances_to_mobile_final_to_predict, on=['ID'],how='left')
# remplacing the NaN values by 400km (the filter in Arcgis was 400km and above for the test set)
to_predict = to_predict.fillna(400)
# remplacing all values above 400km by 400km
to_predict['KM_to_mobile'] = to_predict['KM_to_mobile'].apply(lambda x : 400 if x>400 else x)

In [10]:
# Load the prediction file again for further need
to_predict2 = pd.read_csv('../input/test.csv',decimal=".")

In [11]:
# loading the infos by regions
info_regions = pd.read_csv('../input/Info_Regions.csv',delimiter=';')
info_regions = info_regions.rename(columns={'Region': 'Region_Nam'})

In [12]:
# Load the regions for training Test (extracted from Arcgis)
Test_Regions = pd.read_csv('../input/Test_Regions.csv',decimal=".")
Test_Regions_reduced = Test_Regions[['ID','Region_Nam']]
Test_Regions_add = pd.merge(Test_Regions_reduced, info_regions, on=['Region_Nam'],how='left')
to_predict = pd.merge(to_predict, Test_Regions_add, on=['ID'],how='left')
#train['Region_Nam'] = train['Region_Nam'].fillna('Unknown')
to_predict = to_predict.fillna(train.mean())

In [13]:
# Load the regions for training Test (extracted from Arcgis)
Train_Regions = pd.read_csv('../input/Train_Regions.csv',decimal=".")
Train_Regions_reduced = Train_Regions[['ID','Region_Nam']]
Train_Regions_add = pd.merge(Train_Regions_reduced, info_regions, on=['Region_Nam'],how='left')
train = pd.merge(train, Train_Regions_add, on=['ID'],how='left')
#train['Region_Nam'] = train['Region_Nam'].fillna('Unknown')
train = train.fillna(train.mean())

In [14]:
def preprocessing(dataset):
    
    ## Feature Engineering
    
    #Q1_1 Age by quantiles : Quantiles ['0-25Q', '25-50Q', '50-75Q', '75-100Q']
    quantile_labels = [1, 2, 3, 4]
    quantile_list = [0, .25, .5, .75, 1.]
    dataset['Q1_1'] = pd.qcut(dataset['Q1'], q=quantile_list, labels=quantile_labels)
    #KM_to_banks by quantiles : Quantiles ['0-25Q', '25-50Q', '50-75Q', '75-100Q']
    quantile_labels = [1, 2, 3, 4]
    quantile_list = [0, .25, .5, .75, 1.]
    dataset['KM_to_banks_1'] = pd.qcut(dataset['KM_to_bank'], q=quantile_list, labels=quantile_labels)
    dataset['KM_to_mobile_1'] = pd.qcut(dataset['KM_to_mobile'], q=quantile_list, labels=quantile_labels)
    dataset['2016 Purchasing Power : Per Capita '] = pd.qcut(dataset['2016 Purchasing Power : Per Capita '], q=quantile_list, labels=quantile_labels)
    dataset['2016 Average Household Size'] = pd.qcut(dataset['2016 Average Household Size'], q=quantile_list, labels=quantile_labels)
    dataset['2016 Population Density (per sq. km)'] = pd.qcut(dataset['2016 Population Density (per sq. km)'], q=quantile_list, labels=quantile_labels)
    dataset['% Bank access within 5km'] = pd.qcut(dataset['% Bank access within 5km'], q=quantile_list, labels=quantile_labels)
    # Q3_1 = 1 for married 0 else
    dataset['Q3_1'] = dataset.apply(lambda x: 1 if x['Q3']==1 else 0,axis=1)
    # Q4_1 = 0 if below or equal some primary, 1 if below or equal some secondary 2 for the rest
    dataset['Q4_1'] = dataset.apply(lambda x: 0 if x['Q4']<=2 else 1 if x['Q4']<=5 else 2,axis=1)
    dataset['Q4_1'] = dataset['Q4_1'].astype('category', ordered=True)
    # Q5_1 = Land ownership (1 has a land 0 no)
    dataset['Q5_1'] = dataset.apply(lambda x: 1 if (x['Q5']==1 or x['Q5']==2 or x['Q6']==1) else 0,axis=1)
    # Q8 = Sources of income (count (removed sources like pension, social, rely on someone))
    dataset['Q8'] = dataset.apply(lambda x: x['Q8_1']+x['Q8_2']+x['Q8_3']+x['Q8_4']+x['Q8_5']+x['Q8_6'],axis=1)
    dataset['Q8'] = dataset['Q8'].astype('category', ordered=True)
    # Q12_1 = Send or receive money (2 = No No, 3:Y/N, 4 : Y-Y)
    dataset['Q12_1'] = dataset.apply(lambda x: x['Q12']+x['Q14'],axis=1)
    dataset['Q12_1'] = dataset['Q12_1'].astype('category', ordered=True)
    # Q13_1 Received money (0 : No, 1 : not in the last 3 months, 2 in the last 3 months : )
    dataset['Q13_1'] = dataset.apply(lambda x : 0 if x['Q13']==-1 else 2 if (x['Q13']==1 or x['Q13']==2 or x['Q13']==3) else 1,axis=1)
    dataset['Q13_1'] = dataset['Q13_1'].astype('category', ordered=True)
    # Q15_1 Received money (0 : No, 1 : not in the last 3 months, 2 in the last 3 months : )
    dataset['Q15_1'] = dataset.apply(lambda x : 0 if x['Q15']==-1 else 2 if (x['Q15']==1 or x['Q15']==2 or x['Q15']==3) else 1,axis=1)
    dataset['Q15_1'] = dataset['Q15_1'].astype('category', ordered=True)
    # Q16_1 Used Mobile_money (0 never, 1 less than monthly, 2 monthly or more)
    dataset['Q16_1'] = dataset.apply(lambda x : 0 if (x['Q16']<=1 and x['Q17']<=1) else 1 if (x['Q16']>=5 or x['Q17']>=5) else 2,axis=1)
    dataset['Q16_1'] = dataset['Q16_1'].astype('category', ordered=True)
    # Q18_1 Literacy Kiswhahili (0 no, 1 middle, 2 read and write)
    dataset['Q18_1'] = dataset.apply(lambda x : 0 if x['Q18']>=4 else 1 if x['Q18']>=2 else 2,axis=1)
    dataset['Q18_1'] = dataset['Q18_1'].astype('category', ordered=True)
    # Q19_1 Literacy English (0 no, 1 middle, 2 read and write)
    dataset['Q19_1'] = dataset.apply(lambda x : 0 if x['Q19']>=4 else 1 if x['Q19']>=2 else 2,axis=1)
    dataset['Q19_1'] = dataset['Q19_1'].astype('category', ordered=True)
    # Q20 Literacy English or Kiswhahili (sum)
    dataset['Q20'] = dataset.apply(lambda x : x['Q18_1']+x['Q19_1'],axis=1)
    dataset['Q20'] = dataset['Q20'].astype('category', ordered=True)
    
    ## Remove columns not used 
    #dataset = dataset.drop(columns=['ID', 'Latitude','Longitude'])
    
    ## Columns as category
    for col in ['Q1', 'Q2','Q3','Q4','Q5','Q6','Q7','Q8_1','Q8_2','Q8_3',
             'Q8_4','Q8_5','Q8_6','Q8_7','Q8_8','Q8_9','Q8_10','Q8_11',
             'Q9','Q10','Q11','Q12','Q13','Q14','Q15','Q16','Q17','Q18','Q19']:
        dataset[col] = dataset[col].astype('category')
    
    return dataset

In [15]:
preprocessing(train)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:39: FutureWarning: specifying 'categories' or 'ordered

,ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,Q8_4,Q8_5,Q8_6,Q8_7,Q8_8,Q8_9,Q8_10,Q8_11,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Latitude,Longitude,mobile_money,savings,borrowing,insurance,mobile_money_classification,KM_to_bank,KM_to_mobile,Region_Nam,2016 Purchasing Power : Per Capita,2016 Average Household Size,2016 Population Density (per sq. km),% Bank access within 5km,Q1_1,KM_to_banks_1,KM_to_mobile_1,Q3_1,Q4_1,Q5_1,Q8,Q12_1,Q13_1,Q15_1,Q16_1,Q18_1,Q19_1,Q20
0,5086,98,2,3,1,1,2,2,0,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,2,-1,2,-1,1,-1,4,4,-4.460442,29.811396,0,0,0,0,0,34.085336,1.110439,Kigoma,1,4,2,1,4,3,2,0,0,1,0,4,0,0,0,0,0,0
1,1258,40,1,1,3,5,1,1,1,0,0,0,0,0,0,0,0,0,0,1,-1,-1,1,4,1,5,4,4,1,4,-6.176438,39.244871,1,1,1,0,3,2.213349,0.533324,Mjini Magharibi,4,3,4,4,3,1,1,1,1,1,1,2,1,1,2,2,0,2
2,331,18,2,4,6,3,2,1,0,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,2,-1,2,-1,1,-1,1,1,-6.825702,37.652798,1,0,0,0,2,1.129646,0.359524,Morogoro,3,2,1,2,1,1,1,0,2,0,0,4,0,0,0,2,2,4
3,6729,50,1,1,3,1,1,1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,-1,1,2,2,-1,4,-1,1,4,-3.372049,35.808307,1,0,1,0,3,14.759292,0.513033,Arusha,4,2,2,4,4,2,1,1,1,1,1,3,2,0,2,2,0,2
4,8671,34,1,1,1,1,2,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,2,-1,1,1,1,-1,1,4,-7.179645,31.039095,1,1,0,1,3,35.708657,3.702071,Rukwa,1,3,2,1,2,3,2,1,0,1,1,3,0,2,0,2,0,2
5,5462,35,1,1,3,3,2,1,0,1,0,1,0,0,0,0,0,0,0,-1,1,-1,2,-1,1,3,1,-1,1,1,-6.362331,37.137741,0,0,1,0,1,62.336336,15.694743,Morogoro,3,2,1,2,2,4,4,1,1,0,2,3,0,2,0,2,2,4
6,4886,31,2,1,3,1,2,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,1,5,1,3,1,-1,1,2,-8.089257,35.836415,1,1,1,1,3,37.746307,29.946015,Iringa,4,1,1,1,2,3,4,1,1,1,1,2,1,2,0,2,1,3
7,621,23,2,4,5,5,2,1,0,0,1,0,0,0,0,0,0,0,0,-1,-1,1,1,4,1,4,5,1,2,2,-8.916028,33.433904,1,1,0,0,3,2.339737,0.921649,Mbeya,4,1,2,4,1,1,1,0,1,0,1,2,1,1,1,1,1,2
8,8302,56,2,3,3,3,2,2,0,1,0,1,0,1,0,0,0,0,0,-1,1,-1,1,5,1,6,5,-1,1,1,-3.972247,32.649951,0,1,1,0,1,16.477870,3.974523,Shinyanga,1,4,3,2,4,2,3,0,1,0,3,2,1,1,1,2,2,4
9,4704,37,2,1,3,3,2,1,0,1,0,0,0,0,0,1,1,0,0,-1,1,-1,1,3,1,4,5,-1,1,4,-8.033973,35.769416,1,0,0,1,3,29.496735,20.680268,Iringa,4,1,1,1,3,3,4,1,1,0,1,2,2,1,1,2,0,2


In [16]:
### Reduce columns
train = train.drop(columns=['ID', 'Latitude','Longitude'])
### Reduce columns
train = train.drop(columns=['Q1','KM_to_bank','KM_to_mobile','Region_Nam'])
train = train.astype('float')

In [17]:
train[['Q3','Q4','Q5','Q9','Q10','Q11']] = train[['Q3','Q4','Q5','Q9','Q10','Q11']].astype('category')

In [18]:
# Initiate Get Dummies
train = pd.get_dummies(train)

In [19]:
train.head()

,Q2,Q6,Q7,Q8_1,Q8_2,Q8_3,Q8_4,Q8_5,Q8_6,Q8_7,Q8_8,Q8_9,Q8_10,Q8_11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,mobile_money,savings,borrowing,insurance,mobile_money_classification,2016 Purchasing Power : Per Capita,2016 Average Household Size,2016 Population Density (per sq. km),% Bank access within 5km,Q1_1,KM_to_banks_1,KM_to_mobile_1,Q3_1,Q4_1,Q5_1,Q8,Q12_1,Q13_1,...,Q4_4.0,Q4_5.0,Q4_6.0,Q4_7.0,Q4_8.0,Q5_1.0,Q5_2.0,Q5_3.0,Q5_4.0,Q5_5.0,Q5_6.0,Q9_-1.0,Q9_1.0,Q9_2.0,Q9_3.0,Q9_5.0,Q9_6.0,Q9_7.0,Q10_-1.0,Q10_1.0,Q10_2.0,Q10_3.0,Q10_4.0,Q10_5.0,Q10_6.0,Q10_7.0,Q10_8.0,Q10_9.0,Q10_10.0,Q11_-1.0,Q11_1.0,Q11_2.0,Q11_3.0,Q11_4.0,Q11_5.0,Q11_6.0,Q11_7.0,Q11_8.0,Q11_9.0,Q11_12.0
0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,-1.0,2.0,-1.0,1.0,-1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,1.0,4.0,3.0,2.0,0.0,0.0,1.0,0.0,4.0,0.0,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,5.0,4.0,4.0,1.0,4.0,1.0,1.0,1.0,0.0,3.0,4.0,3.0,4.0,4.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,-1.0,2.0,-1.0,1.0,-1.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,3.0,2.0,1.0,2.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,4.0,0.0,...,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,-1.0,4.0,-1.0,1.0,4.0,1.0,0.0,1.0,0.0,3.0,4.0,2.0,2.0,4.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,1.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-1.0,1.0,1.0,1.0,-1.0,1.0,4.0,1.0,1.0,0.0,1.0,3.0,1.0,3.0,2.0,1.0,2.0,3.0,2.0,1.0,0.0,1.0,1.0,3.0,0.0,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [20]:
train.shape

(7094, 92)

In [21]:
train.columns = train.columns.str.replace(".0", "")

In [22]:
train = train.astype('int')

In [23]:
# Split Y and X
train_features = train[train.columns.difference(['mobile_money','savings','borrowing','insurance', 'mobile_money_classification'])]
train_label = train.loc[:,'mobile_money_classification']
train_labels = train.loc[:,['mobile_money','savings','borrowing','insurance', 'mobile_money_classification']]
train_labels.head()

,mobile_money,savings,borrowing,insurance,mobile_money_classification
0,0,0,0,0,0
1,1,1,1,0,3
2,1,0,0,0,2
3,1,0,1,0,3
4,1,1,0,1,3


In [24]:
test_size = 0.30
seed = 7
score = 'accuracy'
scoring = {'accuracy': 'accuracy',
           'recall': 'precision_macro',
           'f1': 'f1'}

In [25]:
# Implementation of different ML Algorithms
def models(X_train, Y_train,score):
    clfs = []
    result = []
    names = []
    clfs.append(('LR', LogisticRegression()))
    clfs.append(('LDA', LinearDiscriminantAnalysis()))
    clfs.append(('KNN', KNeighborsClassifier()))
    clfs.append(('CART', DecisionTreeClassifier()))
    clfs.append(('NB', GaussianNB()))
    clfs.append(('SVM', SVC()))
    for algo_name, clf in clfs:
        k_fold = model_selection.KFold(n_splits=10, random_state=seed)
        cv_score = model_selection.cross_val_score(clf, X_train, Y_train, cv=k_fold, scoring=score)
        #result = "%s: %f (%f)" % (algo_name, cv_score.mean(), cv_score.std())
        result.append((algo_name,cv_score.mean(), cv_score.std()))
        names.append(algo_name)
    return (result)

In [26]:
X_train_all, X_test_all, Y_train_all, Y_test_all = model_selection.train_test_split(train_features, train_label, test_size=test_size, random_state=seed)

In [27]:
models(X_train_all, Y_train_all, score)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed

[('LR', 0.6616318881028104, 0.01618935559469984),
 ('LDA', 0.6475356169273706, 0.015577795463301002),
 ('KNN', 0.6153112221717401, 0.013194526484297175),
 ('CART', 0.5621401797884079, 0.021129022731534684),
 ('NB', 0.37241554163691826, 0.03726551715907846),
 ('SVM', 0.6606217952878561, 0.013209782631218758)]

In [28]:
# Evaluation of the Classifier 
# Predictions on test dataset
svm = SVC()
svm.fit(X_train_all, Y_train_all)
pred = svm.predict(X_test_all)
print(accuracy_score(Y_test_all, pred))
print(confusion_matrix(Y_test_all, pred))
print(classification_report(Y_test_all, pred))

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.6712071395021136
[[148 231   0  12]
 [101 395   0  40]
 [  8  25   0 211]
 [ 16  56   0 886]]
              precision    recall  f1-score   support

           0       0.54      0.38      0.45       391
           1       0.56      0.74      0.64       536
           2       0.00      0.00      0.00       244
           3       0.77      0.92      0.84       958

   micro avg       0.67      0.67      0.67      2129
   macro avg       0.47      0.51      0.48      2129
weighted avg       0.59      0.67      0.62      2129



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
# MLP with automatic validation set
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [30]:
# DataSet for deeplearning
train_deepl = train_features.values
train_label_deepl = train_label.values

In [31]:
X_train, X_test, y_train, y_test = train_test_split(train_deepl,train_label_deepl)

In [32]:
train.shape

(7094, 92)

In [33]:
train_deepl.shape

(7094, 87)

In [34]:
train_label_deepl.shape

(7094,)

In [35]:
# Set Seed
seed = 7
np.random.seed(seed)

In [36]:
# Label encode Class (Species)
encoder = LabelEncoder()
encoder.fit(train_label_deepl)
encoded_Y = encoder.transform(train_label_deepl)
# One Hot Encode
y_dummy = np_utils.to_categorical(encoded_Y)

In [37]:
y_dummy.shape

(7094, 4)

In [38]:
# Deep Learnig Function
def deepml_model():
    # Model Creation
    deepml = Sequential()
    deepml.add(Dense(8, input_dim=87, activation='relu'))
    deepml.add(Dense(8, input_dim=8, activation='relu'))
    deepml.add(Dense(4, activation='softmax'))
    # Model Compilation
    deepml.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return deepml

In [39]:
estimate = KerasClassifier(build_fn=deepml_model, epochs=200, batch_size=5, verbose=0)

In [40]:
X_train.shape

(5320, 87)

In [41]:
y_train.shape

(5320,)

In [42]:
X_test.shape

(1774, 87)

In [43]:
y_test.shape

(1774,)

In [44]:
X_test

array([[1, 4, 1, ..., 0, 0, 0],
       [3, 1, 2, ..., 0, 0, 0],
       [2, 3, 1, ..., 0, 0, 0],
       ...,
       [2, 1, 3, ..., 0, 0, 0],
       [2, 2, 1, ..., 0, 0, 0],
       [2, 4, 3, ..., 0, 0, 0]])

In [45]:
y_test

array([2, 1, 0, ..., 3, 3, 2])

In [46]:
preprocessing(to_predict)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:39: FutureWarning: specifying 'categories' or 'ordered

,ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,Q8_4,Q8_5,Q8_6,Q8_7,Q8_8,Q8_9,Q8_10,Q8_11,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Latitude,Longitude,KM_to_bank,KM_to_mobile,Region_Nam,2016 Purchasing Power : Per Capita,2016 Average Household Size,2016 Population Density (per sq. km),% Bank access within 5km,Q1_1,KM_to_banks_1,KM_to_mobile_1,Q3_1,Q4_1,Q5_1,Q8,Q12_1,Q13_1,Q15_1,Q16_1,Q18_1,Q19_1,Q20
0,2352,30,2,1,3,6,2,1,0,0,1,0,0,0,0,0,0,0,0,-1,-1,1,1,6,1,4,5,-1,1,4,-2.596421,32.913584,1.583605,0.457470,Mwanza,3.0,4.0,4.0,4.0,2,1,1,1,1,0,1,2,1,1,1,2,0,2
1,8208,53,1,1,3,1,2,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,2,-1,1,4,1,-1,1,4,-1.778303,34.062640,41.846446,7.504929,Mara,2.0,3.0,3.0,3.0,4,4,3,1,1,1,1,3,0,1,0,2,0,2
2,2785,23,2,1,3,5,1,1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,-1,1,4,1,4,1,-1,1,1,-1.851789,31.536866,12.382164,0.535899,Kagera,2.0,2.0,4.0,3.0,1,2,1,1,1,1,1,2,1,1,0,2,2,4
3,2967,30,2,3,3,4,1,2,0,1,0,0,0,0,0,0,1,0,0,-1,1,-1,2,-1,2,-1,1,-1,1,4,-1.235656,34.229012,20.238699,8.564699,Mara,2.0,3.0,3.0,3.0,2,2,3,0,1,1,1,4,0,0,0,2,0,2
4,1697,28,2,1,5,4,2,1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,-1,1,4,1,6,4,3,1,2,-7.780934,35.691878,0.859776,0.303721,Iringa,4.0,1.0,1.0,1.0,2,1,1,1,1,0,1,2,1,1,2,2,1,3
5,4794,52,2,1,3,3,2,1,0,1,0,0,0,0,0,0,0,0,0,-1,2,-1,2,-1,1,4,1,-1,1,2,-5.064700,34.904008,32.002994,7.859645,Singida,1.0,3.0,1.0,3.0,4,3,3,1,1,0,1,3,0,1,0,2,1,3
6,2675,17,2,4,3,3,2,1,1,0,0,0,0,0,0,0,0,0,0,3,-1,-1,2,-1,2,-1,4,-1,1,1,-7.681534,36.039689,39.036109,1.250339,Iringa,4.0,1.0,1.0,1.0,1,3,2,0,1,0,1,4,0,0,2,2,2,4
7,2989,30,2,1,3,3,1,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,1,5,1,3,5,-1,1,4,-2.505206,32.009659,47.373423,0.987244,Geita,2.0,4.0,4.0,2.0,2,4,1,1,1,1,1,2,1,2,1,2,0,2
8,1634,42,2,1,6,2,2,1,0,0,0,1,1,0,0,0,0,0,0,-1,-1,-1,2,-1,2,-1,1,1,1,1,-6.125217,39.219710,4.361879,0.659425,Mjini Magharibi,4.0,3.0,4.0,4.0,3,1,1,1,2,1,2,4,0,0,0,2,2,4
9,4147,45,2,1,3,1,2,1,0,1,0,1,0,0,0,0,0,0,0,-1,1,-1,2,-1,1,3,1,-1,1,4,-4.112269,34.821871,49.610059,6.123250,Singida,1.0,3.0,1.0,3.0,3,4,3,1,1,1,2,3,0,2,0,2,0,2


In [47]:
### Reduce columns
to_predict = to_predict.drop(columns=['ID', 'Latitude','Longitude'])
### Reduce columns
to_predict = to_predict.drop(columns=['Q1','KM_to_bank','KM_to_mobile','Region_Nam'])
to_predict = to_predict.astype('float')

In [48]:
to_predict.head()

,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,Q8_4,Q8_5,Q8_6,Q8_7,Q8_8,Q8_9,Q8_10,Q8_11,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,2016 Purchasing Power : Per Capita,2016 Average Household Size,2016 Population Density (per sq. km),% Bank access within 5km,Q1_1,KM_to_banks_1,KM_to_mobile_1,Q3_1,Q4_1,Q5_1,Q8,Q12_1,Q13_1,Q15_1,Q16_1,Q18_1,Q19_1,Q20
0,2.0,1.0,3.0,6.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,1.0,1.0,6.0,1.0,4.0,5.0,-1.0,1.0,4.0,3.0,4.0,4.0,4.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,1.0,1.0,1.0,2.0,0.0,2.0
1,1.0,1.0,3.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,-1.0,2.0,-1.0,1.0,4.0,1.0,-1.0,1.0,4.0,2.0,3.0,3.0,3.0,4.0,4.0,3.0,1.0,1.0,1.0,1.0,3.0,0.0,1.0,0.0,2.0,0.0,2.0
2,2.0,1.0,3.0,5.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,4.0,1.0,4.0,1.0,-1.0,1.0,1.0,2.0,2.0,4.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,2.0,2.0,4.0
3,2.0,3.0,3.0,4.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,1.0,-1.0,2.0,-1.0,2.0,-1.0,1.0,-1.0,1.0,4.0,2.0,3.0,3.0,3.0,2.0,2.0,3.0,0.0,1.0,1.0,1.0,4.0,0.0,0.0,0.0,2.0,0.0,2.0
4,2.0,1.0,5.0,4.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,4.0,1.0,6.0,4.0,3.0,1.0,2.0,4.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,3.0


In [49]:
to_predict[['Q3','Q4','Q5','Q9','Q10','Q11']] = to_predict[['Q3','Q4','Q5','Q9','Q10','Q11']].astype('category')

In [50]:
to_predict.shape

(2365, 46)

In [51]:
to_predict = pd.get_dummies(to_predict)

In [52]:
to_predict.columns

Index(['Q2', 'Q6', 'Q7', 'Q8_1', 'Q8_2', 'Q8_3', 'Q8_4', 'Q8_5', 'Q8_6',
       'Q8_7', 'Q8_8', 'Q8_9', 'Q8_10', 'Q8_11', 'Q12', 'Q13', 'Q14', 'Q15',
       'Q16', 'Q17', 'Q18', 'Q19', '2016 Purchasing Power : Per Capita ',
       '2016 Average Household Size', '2016 Population Density (per sq. km)',
       '% Bank access within 5km', 'Q1_1', 'KM_to_banks_1', 'KM_to_mobile_1',
       'Q3_1', 'Q4_1', 'Q5_1', 'Q8', 'Q12_1', 'Q13_1', 'Q15_1', 'Q16_1',
       'Q18_1', 'Q19_1', 'Q20', 'Q3_1.0', 'Q3_2.0', 'Q3_3.0', 'Q3_4.0',
       'Q4_1.0', 'Q4_2.0', 'Q4_3.0', 'Q4_4.0', 'Q4_5.0', 'Q4_6.0', 'Q4_7.0',
       'Q4_8.0', 'Q5_1.0', 'Q5_2.0', 'Q5_3.0', 'Q5_4.0', 'Q5_5.0', 'Q5_6.0',
       'Q9_-1.0', 'Q9_1.0', 'Q9_2.0', 'Q9_3.0', 'Q9_4.0', 'Q9_5.0', 'Q9_6.0',
       'Q9_7.0', 'Q10_-1.0', 'Q10_1.0', 'Q10_2.0', 'Q10_3.0', 'Q10_4.0',
       'Q10_5.0', 'Q10_6.0', 'Q10_7.0', 'Q10_8.0', 'Q10_9.0', 'Q10_10.0',
       'Q11_-1.0', 'Q11_1.0', 'Q11_3.0', 'Q11_4.0', 'Q11_5.0', 'Q11_7.0',
       'Q11_8.0', 'Q11

In [53]:
to_predict.columns = to_predict.columns.str.replace(".0", "")

In [54]:
missing_columns = (set(train_features.columns) - set(to_predict.columns))
for i in missing_columns:
        to_predict[i] = 0

In [55]:
missing_columns

{'Q11_2', 'Q11_6'}

In [56]:
additional_columns = (set(to_predict.columns) - set(train_features.columns))
for i in additional_columns:
    to_predict = to_predict.drop(columns=i)

In [57]:
additional_columns

{'Q9_4'}

In [58]:
set(to_predict.columns)-set(train.columns)

set()

In [59]:
to_predict.columns

Index(['Q2', 'Q6', 'Q7', 'Q8_1', 'Q8_2', 'Q8_3', 'Q8_4', 'Q8_5', 'Q8_6',
       'Q8_7', 'Q8_8', 'Q8_9', 'Q8_', 'Q8_11', 'Q12', 'Q13', 'Q14', 'Q15',
       'Q16', 'Q17', 'Q18', 'Q19', '16 Purchasing Power : Per Capita ',
       '16 Average Household Size', '16 Population Density (per sq. km)',
       '% Bank access within 5km', 'Q1_1', 'KM_to_banks_1', 'KM_to_mobile_1',
       'Q3_1', 'Q4_1', 'Q5_1', 'Q8', 'Q12_1', 'Q13_1', 'Q15_1', 'Q16_1',
       'Q18_1', 'Q19_1', 'Q', 'Q3_1', 'Q3_2', 'Q3_3', 'Q3_4', 'Q4_1', 'Q4_2',
       'Q4_3', 'Q4_4', 'Q4_5', 'Q4_6', 'Q4_7', 'Q4_8', 'Q5_1', 'Q5_2', 'Q5_3',
       'Q5_4', 'Q5_5', 'Q5_6', 'Q9_-1', 'Q9_1', 'Q9_2', 'Q9_3', 'Q9_5', 'Q9_6',
       'Q9_7', 'Q_-1', 'Q_1', 'Q_2', 'Q_3', 'Q_4', 'Q_5', 'Q_6', 'Q_7', 'Q_8',
       'Q_9', 'Q_', 'Q11_-1', 'Q11_1', 'Q11_3', 'Q11_4', 'Q11_5', 'Q11_7',
       'Q11_8', 'Q11_9', 'Q11_12', 'Q11_6', 'Q11_2'],
      dtype='object')

In [60]:
train_features.columns

Index(['% Bank access within 5km', '16 Average Household Size',
       '16 Population Density (per sq. km)',
       '16 Purchasing Power : Per Capita ', 'KM_to_banks_1', 'KM_to_mobile_1',
       'Q', 'Q11_-1', 'Q11_1', 'Q11_12', 'Q11_2', 'Q11_3', 'Q11_4', 'Q11_5',
       'Q11_6', 'Q11_7', 'Q11_8', 'Q11_9', 'Q12', 'Q12_1', 'Q13', 'Q13_1',
       'Q14', 'Q15', 'Q15_1', 'Q16', 'Q16_1', 'Q17', 'Q18', 'Q18_1', 'Q19',
       'Q19_1', 'Q1_1', 'Q2', 'Q3_1', 'Q3_1', 'Q3_2', 'Q3_3', 'Q3_4', 'Q4_1',
       'Q4_1', 'Q4_2', 'Q4_3', 'Q4_4', 'Q4_5', 'Q4_6', 'Q4_7', 'Q4_8', 'Q5_1',
       'Q5_1', 'Q5_2', 'Q5_3', 'Q5_4', 'Q5_5', 'Q5_6', 'Q6', 'Q7', 'Q8', 'Q8_',
       'Q8_1', 'Q8_11', 'Q8_2', 'Q8_3', 'Q8_4', 'Q8_5', 'Q8_6', 'Q8_7', 'Q8_8',
       'Q8_9', 'Q9_-1', 'Q9_1', 'Q9_2', 'Q9_3', 'Q9_5', 'Q9_6', 'Q9_7', 'Q_',
       'Q_-1', 'Q_1', 'Q_2', 'Q_3', 'Q_4', 'Q_5', 'Q_6', 'Q_7', 'Q_8', 'Q_9'],
      dtype='object')

In [61]:
to_predict = to_predict.sort_index(axis=1)

In [62]:
train = train.sort_index(axis=1)

In [63]:
to_predict.head()

,% Bank access within 5km,16 Average Household Size,16 Population Density (per sq. km),16 Purchasing Power : Per Capita,KM_to_banks_1,KM_to_mobile_1,Q,Q11_-1,Q11_1,Q11_12,Q11_2,Q11_3,Q11_4,Q11_5,Q11_6,Q11_7,Q11_8,Q11_9,Q12,Q12_1,Q13,Q13_1,Q14,Q15,Q15_1,Q16,Q16_1,Q17,Q18,Q18_1,Q19,Q19_1,Q1_1,Q2,Q3_1,Q3_1,Q3_2,Q3_3,Q3_4,Q4_1,...,Q4_8,Q5_1,Q5_1,Q5_2,Q5_3,Q5_4,Q5_5,Q5_6,Q6,Q7,Q8,Q8_,Q8_1,Q8_11,Q8_2,Q8_3,Q8_4,Q8_5,Q8_6,Q8_7,Q8_8,Q8_9,Q9_-1,Q9_1,Q9_2,Q9_3,Q9_5,Q9_6,Q9_7,Q_,Q_-1,Q_1,Q_2,Q_3,Q_4,Q_5,Q_6,Q_7,Q_8,Q_9
0,4.0,4.0,4.0,3.0,1.0,1.0,2.0,0,1,0,0,0,0,0,0,0,0,0,1.0,2.0,6.0,1.0,1.0,4.0,1.0,5.0,1.0,-1.0,1.0,2.0,4.0,0.0,2.0,2.0,1.0,1,0,0,0,1.0,...,0,0,0.0,0,0,0,0,1,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,3.0,3.0,3.0,2.0,4.0,3.0,2.0,1,0,0,0,0,0,0,0,0,0,0,2.0,3.0,-1.0,0.0,1.0,4.0,1.0,1.0,0.0,-1.0,1.0,2.0,4.0,0.0,4.0,1.0,1.0,1,0,0,0,1.0,...,0,1,1.0,0,0,0,0,0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,3.0,2.0,4.0,2.0,2.0,1.0,4.0,1,0,0,0,0,0,0,0,0,0,0,1.0,2.0,4.0,1.0,1.0,4.0,1.0,1.0,0.0,-1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1,0,0,0,1.0,...,0,0,1.0,0,0,0,1,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,3.0,3.0,3.0,2.0,2.0,3.0,2.0,1,0,0,0,0,0,0,0,0,0,0,2.0,4.0,-1.0,0.0,2.0,-1.0,0.0,1.0,0.0,-1.0,1.0,2.0,4.0,0.0,2.0,2.0,0.0,0,0,1,0,1.0,...,0,0,1.0,0,0,1,0,0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,1.0,1.0,1.0,4.0,1.0,1.0,3.0,1,0,0,0,0,0,0,0,0,0,0,1.0,2.0,4.0,1.0,1.0,6.0,1.0,4.0,2.0,3.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1,0,0,0,1.0,...,0,0,0.0,0,0,1,0,0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [64]:
to_predict = to_predict.fillna(0)

In [65]:
to_predict = to_predict.astype('int')

In [66]:
to_predict_array = to_predict.values

In [67]:
STOP

NameError: name 'STOP' is not defined

In [68]:
estimate_1 = KerasClassifier(build_fn=deepml_model, epochs=30, batch_size=5, verbose=10)

In [69]:
estimate_1.fit(X_train,y_train)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [70]:
test_pred = estimate_1.predict(X_test)

In [71]:
test_pred

array([3, 1, 1, ..., 3, 3, 3])

In [72]:
y_test

array([2, 1, 0, ..., 3, 3, 2])

In [73]:
Y_final_df = pd.DataFrame([y_test,test_pred])

In [74]:
Y_final_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1734,1735,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765,1766,1767,1768,1769,1770,1771,1772,1773
0,2,1,0,3,3,1,3,1,3,1,3,1,2,1,1,0,3,2,2,0,3,2,3,3,2,3,3,3,3,3,3,0,3,3,3,0,2,3,2,3,...,2,2,2,0,2,3,3,3,3,3,1,3,3,0,3,3,1,2,3,3,1,3,3,3,2,3,2,0,0,1,3,3,3,3,1,3,3,3,3,2
1,3,1,1,3,3,3,3,1,3,1,0,1,1,1,0,0,3,0,3,1,3,3,3,3,3,1,3,1,3,3,3,0,3,3,3,0,3,3,3,3,...,3,3,1,1,1,3,3,3,3,3,0,3,3,0,3,3,1,3,3,1,1,3,3,3,3,3,3,1,3,0,3,3,3,3,1,3,3,3,3,3


In [75]:
Y_final_df.shape

(2, 1774)

In [76]:
Y_final_df = Y_final_df.transpose()

In [77]:
Y_final_df.head()

,0,1
0,2,3
1,1,1
2,0,1
3,3,3
4,3,3


In [78]:
Y_final_df['Diff'] = Y_final_df[1]-Y_final_df[0]

In [79]:
Y_final_df['Diff'].sum()/Y_final_df['Diff'].count()

0.04735062006764374

In [80]:
preds = estimate_1.predict(to_predict_array)

In [81]:
preds

array([3, 3, 3, ..., 1, 1, 0])

In [82]:
predictions = estimate_1.predict_proba(to_predict_array)

In [83]:
predictions = pd.DataFrame(predictions)

In [84]:
# Rename the columns
predictions.columns = ['no_financial_services','other_only','mm_only','mm_plus']
# Add the correct index (ID)
predictions['ID'] = to_predict2['ID']
predictions = predictions.set_index('ID')
predictions.head()

,no_financial_services,other_only,mm_only,mm_plus
ID,,,,
2352,0.000186,0.000476,0.173968,0.825369
8208,0.017634,0.040374,0.075304,0.866689
2785,0.001170,0.003770,0.093120,0.901940
2967,0.470890,0.442721,0.026841,0.059548
1697,0.000555,0.001882,0.068506,0.929057


In [85]:
predictions.to_csv('predictions_deeplearning_1.csv')

In [86]:
STOP

NameError: name 'STOP' is not defined